# Tutorial: A 3D Point Cloud Drawing Tool Using Cyclic Coordinate Descent (Wren McQueary)
Final project submission for CS 685 at George Mason University.

## Problems addressed and motivation
In class, we already discussed how inverse kinematics (IK) can be used to make a robot arm place its end effector at a desired position.  We learned an analytical approach to solving IK for a 2-arm robot in 2D.  We also learned how to use the MoveIt tool to run IK for more complicated arms in 3D.  However, after hours of trying to install MoveIt on my local Ubuntu virtual machine, including significant time with Prof. Stein's help, we gave up on installing it.  This disappointed me, since MoveIt would have opened up many creative possibilities.

I love using engineering tools to create art; this was a large part of why I worked in STEM museum education for a few years.  So even without access to MoveIt, I chose to create a simulated robot arm which can "paint" points in 3D space at the location of its end effector.  This tool can create artistic and colorful point clouds in 3D space.  It also avoids collision with obstacles.

## Datasets used/collected
I did not access existing datasets for this project.  However, I still referred to existing IK algorithms.  These are discussed in the *Existing Work* section.

## Existing work
3D inverse kinematics for robot arms is well studied and many approaches exist for implementing it.  In this section, I highlight some key approaches.


[Cyclic Coordinate Descent](https://motion.cs.illinois.edu/RoboticSystems/InverseKinematics.html#Cyclic-Coordinate-Descent) entails initializing all angles at arbitrary values, then visiting each angle one-by-one.  For each angle visited, that angle is set to whatever value would minimize the distance between the end effector and the goal, assuming all other angles are held constant.  This is repeated for some number of cycles until the end effector is sufficiently close to the goal.


[Niels Joubert from UC Berkeley describes](https://inst.eecs.berkeley.edu/~cs184/fa09/resources/ik.pdf) how many approaches hinge on finding the Jacobian of the error in each dimension with respect to the angle losses, then descending it in the style of gradient descent.  This is different from Cyclic Coordinate Descent in that it considers error in each dimension separately and also adjusts all angles simultaneously.  Joubert provides a useful figure highlighting how the Jacobian is iteratively descended:

![jacobian_flowchart](tutorial_images/jacobian_flowchart.png)


Collision avoidance is a key concern in real-world applications of robot arm IK.  An arm must avoid colliding with itself and with obstacles in its environment, some of which may be moving.  Furthermore, in situations where multiple arms are used nearby one another, they must avoid collisions with each other.  [Zhao et al](https://link.springer.com/article/10.1007/s10846-020-01279-w) published an approach for solving IK problems with multiple arms in environments where objects may be moving.  They build their work ground-up from Newtonian principles and first-order differential equations.


[Park et al](https://link-springer-com.mutex.gmu.edu/content/pdf/10.1007/s12555-019-0076-7.pdf) created another system for multiple arms in close proximity within a dynamic environment.  In addition to using Jacobians as described previously, they also leveraged a potential field, which they claim is frequently seen in IK solutions for dynamic environments.


[Chua et al](https://ieeexplore.ieee.org/document/6628553) published a paper in which they designed an IK system for a humanoid robot.  Their approach minimizes energy usage and once again avoids self-collision.

## Technical details of my method
I built a Cyclic Coordinate Descent (CCD) system for a four-segment arm.  Much like a construction crane, the arm's base angle $\theta_0$ about the vertical axis, and its three other joints rotate about the line parallel to the ground.  The base angle $\theta_0$ points in the $+x$ direction when set to 0, and increases in the counterclockwise direction when viewed from above.  The next angle $\theta_1$ points horizontally when set to 0, and increases in the counterclockwise direction when viewed from the robot's right.  The remaining angles $\theta_2, \theta_3$ point in the same direction as the angle immediately before them when set to 0, and increase in the counterclockwise direction when viewed from the robot's left.  A single obstacle exists near the robot, in the form of a 2x2 rectangle aligned with the $y=1$ plane and centered on $(0, 1, 0)$.  Both forward and inverse kinematics are powered by matrix multiplications of affine transformations, as done in class.

The system takes in a series of *goal points* to draw, as well as an optional color for each point.  The robot begins with all its angles set to 0.  For each goal point, it uses CCD (described in *Existing work*) to find angles that bring it "closest" to the goal.  To avoid collision, angles which would reuslt in a collision with an obstacle are not considered.  CCD is not an optimal approach, but as is discussed in the section *Performance metrics, results, discussion*, it does decently well.

At this point, some visualization code I've written takes over.  Angles for a series of inbetween poses are found for each pair of consecutive steps, by taking graduated linear combinations of the new and old values per angle.  Then a `matplotlib` animation shows the robot moving from position to position.  In this animation, the inbetween poses serve to approximate how the arm might move.  However, they are for aesthetic purposes only, and do not guarantee collision avoidance.  That guarantee is only given in the goal frames, to which the inbetween frames are a less rigorous mesh.

## Performance metrics, results, discussion
To determine how accurately the robot places its end effector, I generated four tests, in the form of three .txt files, each containing a series of points for it to draw.  These files are as follows:
- `breadth_test.txt`, in which points surround the base and are at various heights
- `smile.txt`, in which all points are coplanar and make up a smiling face in front of the obstacle
- `obstacle_avoidance_test.txt`, in which all points are on the opposite side of the obstacle, and none are within the base's line of sight

Below, I show how the arm performed on each test.  Each test is accompanied with the following pieces of information:
- A figure, showing the goal points (black rings), and the locations of the actual poitns painted by the robot (colorful circles).
- The mean distance between each goal point and the corresponding point painted by the robot, measured in arm segment lengths.
- The standard deviation of that distance, measured in arm segment lengths.


### Breadth test

![breadth_test](tutorial_images/breadth_test.png)

`mean = 0.0050, std = 0.0015`

The arm performed extremely well on this test.  Its mean error was low and its performance was consistent.


### Smile

![smile](tutorial_images/smile.png)

`mean = 0.0057, std = 0.0028`

The arm performed extremely well on this test, doing comparably well to the breadth test.


### Smile with obstacle avoidance

![smile_with_obstacle_avoidance](tutorial_images/smile_with_obstacle_avoidance.png)

`mean = 0.0325, std = 0.0612`

Enabling obstacle avoidance forced the arm to contort in order to reach certain obstacles.  In particular it struggled to paint the eyes of the smile because of how close they are to the obstacle.


### Obstacle avoidance test with obstacle avoidance

![obstacle_avoidance_test_with_obstacle_avoidance](tutorial_images/obstacle_avoidance_test_with_obstacle_avoidance.png)

`mean = 0.9848, std = 0.0320`

The arm performed badly on this test.  It painted all its points within the base's line of sight.  This happened because the arm is initialized at $\theta_1 = 0$, so any increase or decrease of the angle beyond a few degrees would bring the end effector *farther* from any goal point; therefore these angle changes were not considered.  Yet changing $\theta_1$ is required because of the arm's crane-like design; it cannot reach around the barrier from the left or right, so it must reach around from above or below instead.


### Obstacle avoidance test with obstacle avoidance, with assistive initialization

![obstacle_avoidance_test_with_obstacle_avoidance_with_assistive_initialization](tutorial_images/obstacle_avoidance_test_with_obstacle_avoidance_with_assistive_initialization.png)

`mean = 0.0299, std = 0.0320`

The failure of the previous test inspired me to try initializing the angles to more agreeable values, to avoid the same CCD stall.  In this new test, I initialized the angles to $[\pi/2, 0, 0, 2\pi/3, \pi/3]$, in that order.  These angles got the CCD algorithm "unstuck", and allowed it to reach a much better accuracy and standard deviation.

## Future work
I showed that assistive initialization can help overcome a stalled CCD process.  This project could be improved by automating assistive initialization, such as by starting the arm pointing directly toward the goal, with all segments straight, and then gradually adjusting the angles so that the end effector stays still but the arm pushes itself away from the center of the obstacle.

As discussed earlier, the inbetweener frames do not guarantee obstacle avoidance, even though the key frames they connect do.  The inbetweener frames could guarantee obstacle avoidance as well if they began as additional goal positions, on each of which another round of obstacle-avoiding CCD were performed.

## References
- Chua, Yuanwei, Keng Peng Tee, and Rui Yan. "Robust optimal inverse kinematics with self-collision avoidance for a humanoid robot." 2013 IEEE RO-MAN. IEEE, 2013.
- Hauser, Kris. “Robotic Systems (Draft).” Book, https://motion.cs.illinois.edu/RoboticSystems/. 
- Joubert, Niels, and U. Berkeley. "Numerical methods for inverse kinematics." Retrieved February 28 (2008): 2015.
- Park, Sun-Oh, Min Cheol Lee, and Jaehyung Kim. "Trajectory planning with collision avoidance for redundant robots using jacobian and artificial potential field-based real-time inverse kinematics." International Journal of Control, Automation and Systems 18.8 (2020): 2095-2107.
- Zhao, Liangliang, Jingdong Zhao, and Hong Liu. "Solving the Inverse Kinematics Problem of Multiple Redundant Manipulators with Collision Avoidance in Dynamic Environments." Journal of Intelligent & Robotic Systems 101.2 (2021): 1-18.

## Try it yourself!
If you want to try the code, see `README.md` for instructions!  It goes over how to create your own custom point cloud with custom colors, as well as how to use the tool's many CLI flags.